定义T = 20, t = 10

In [32]:
T = 20
t = 10

随机一个V

In [33]:
import numpy as np

V = np.copy(np.random.random(T-t+1) * 10)
V

array([4.54549803, 4.37424021, 1.35832777, 2.13420587, 4.62398444,
       5.23736648, 8.87011339, 4.64115472, 7.71809923, 0.32857495,
       0.19143964])

计算C(T, t)

In [34]:
C = 1 + (1 - np.e**(-T+t-1)) / (1 - 1 / np.e)
C

2.581950285167711

定义t到T的下标i

In [66]:
i = np.arange(t, T+1)
i

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

定义目标函数，scipy中最优化的是最小值，所以和原式相比取了一个负号

In [36]:
fun = lambda x: -np.sum((V-T+i-np.log(C*x/(1-np.sum(x))))*x)

定义x1自身的约束条件，下限是0，没有上限

In [37]:
bnds = [(0, None) for i in range(T-t+1)]
bnds

[(0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None),
 (0, None)]

定义x1的约束条件，x1的和小于1，scipy中最优化设约束时是函数值大于0，所以此处为1 - np.sum(x)

In [38]:
cons = {'type': 'ineq', 'fun': lambda x:  1-np.sum(x)}

初始化x1

In [39]:
x1_0 = np.zeros(T-t+1) + 1e-3
x1_0

array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001])

优化x1

In [40]:
from scipy.optimize import minimize

res = minimize(fun, x1_0, method='SLSQP', bounds=bnds,
               constraints=cons, options={'disp': True})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -3.0350092200841488
            Iterations: 65
            Function evaluations: 927
            Gradient evaluations: 65


d:\python36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
d:\python36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
d:\python36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


优化后的值

In [41]:
res.x

array([7.79288179e-06, 1.71216033e-05, 3.13053403e-06, 1.30150539e-05,
       4.32823722e-04, 2.15302848e-03, 2.21289179e-01, 8.75841086e-03,
       5.16594729e-01, 8.70372469e-04, 2.04810012e-03])

上一步中x都大于0，满足第一个约束条件，这一步中，x的和小于1，满足第二个约束条件

In [42]:
np.sum(res.x)

0.7521877042709685

目标函数的值

In [43]:
-fun(res.x)

3.0350092200841488

取出来x1

In [44]:
x1 = res.x
x1

array([7.79288179e-06, 1.71216033e-05, 3.13053403e-06, 1.30150539e-05,
       4.32823722e-04, 2.15302848e-03, 2.21289179e-01, 8.75841086e-03,
       5.16594729e-01, 8.70372469e-04, 2.04810012e-03])

计算xtT2

In [45]:
xtT2 = (1 - np.sum(x1)) / C
xtT2

0.09597872474641192

计算p

In [46]:
p = V-T+i-np.log(C*x1/(1-np.sum(x1)))
p

array([3.96416914, 4.00578102, 3.68900599, 4.03998101, 4.02553574,
       4.03461758, 4.03476959, 4.03526679, 4.03496711, 4.03153553,
       4.0386536 ])

根据x1计算Δx2

In [47]:
Δx2 = -(x1 + xtT2 * np.exp(-T+i))
Δx2

array([-1.21503092e-05, -2.89663189e-05, -3.53278093e-05, -1.00536322e-04,
       -6.70731195e-04, -2.79972804e-03, -2.23047091e-01, -1.35369102e-02,
       -5.29584037e-01, -3.61789721e-02, -9.80268249e-02])

因为x(t,t-1,2)等于1，所以根据Δx2可以求出x2

In [48]:
x2 = np.array([1 + np.sum(Δx2[:i]) for i in range(1, 12)])
x2

array([0.99998785, 0.99995888, 0.99992356, 0.99982302, 0.99915229,
       0.99635256, 0.77330547, 0.75976856, 0.23018452, 0.19400555,
       0.09597872])

根据x2计算I

In [49]:
I = np.array([1 + np.sum(Δx2[:i]) for i in range(11)])
I

array([1.        , 0.99998785, 0.99995888, 0.99992356, 0.99982302,
       0.99915229, 0.99635256, 0.77330547, 0.75976856, 0.23018452,
       0.19400555])

根据I, x1, x2计算x0

In [50]:
x0 = I - x1 - x2
x0

array([4.35742736e-06, 1.18447156e-05, 3.21972752e-05, 8.75212682e-05,
       2.37907473e-04, 6.46699560e-04, 1.75791166e-03, 4.77849933e-03,
       1.29893079e-02, 3.53085996e-02, 9.59787247e-02])

验证一下是不是真的是最优解，加个偏差项看看

In [57]:
δx1 = np.diag(np.ones(T-t+1) * 1e-3)
δx1

array([[0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.001, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0. 

x1任意一个维度变动0.001目标函数都会比scipy最优解(最小值)大，所以在精确度0.001的条件下，scipy求出的解至少是个局部最优解

In [71]:
for j in range(11):
    
    print(fun(x1 + δx1[j]) > fun(x1))

True
True
True
True
True
True
True
True
True
True
True
